In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("../data/processed/churn_segmented.csv")

In [3]:
import joblib
from sklearn.model_selection import train_test_split

In [4]:
X = df.drop(['Churn', 'segment'], axis=1)
y = df['Churn']

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [6]:
xgb = joblib.load("../models/xgboost_churn_model.pkl")

In [7]:
df.loc[X_test.index, 'churn_prob'] = xgb.predict_proba(X_test)[:, 1]

In [8]:
df['churn_prob'] = df['churn_prob'].fillna(0)

In [9]:
df['baseline_expected_revenue'] = (df['MonthlyCharges'] * (1 - df['churn_prob']))

In [10]:
df['discount_rate'] = 0.0

df.loc[df['segment'] == 1, 'discount_rate'] = 0.10   # price-sensitive
df.loc[df['segment'] == 2, 'discount_rate'] = 0.05   # new customers

In [11]:
df['churn_reduction_factor'] = 0.0

df.loc[df['discount_rate'] == 0.10, 'churn_reduction_factor'] = 0.30
df.loc[df['discount_rate'] == 0.05, 'churn_reduction_factor'] = 0.15

In [12]:
df['adjusted_churn_prob'] = (df['churn_prob'] * (1 - df['churn_reduction_factor']))

In [13]:
df['adjusted_monthly_charge'] = (df['MonthlyCharges'] * (1 - df['discount_rate']))

In [14]:
df['post_strategy_expected_revenue'] = (df['adjusted_monthly_charge'] * (1 - df['adjusted_churn_prob']))

In [15]:
baseline_revenue = df['baseline_expected_revenue'].sum()
post_strategy_revenue = df['post_strategy_expected_revenue'].sum()

In [16]:
revenue_uplift = post_strategy_revenue - baseline_revenue
uplift_pct = revenue_uplift / baseline_revenue * 100

In [17]:
print(f"Baseline Revenue: ${baseline_revenue:,.2f}")
print(f"Post-Strategy Revenue: ${post_strategy_revenue:,.2f}")
print(f"Revenue Uplift: ${revenue_uplift:,.2f} ({uplift_pct:.2f}%)")

Baseline Revenue: $429,117.53
Post-Strategy Revenue: $411,510.14
Revenue Uplift: $-17,607.40 (-4.10%)


In [18]:
segment_impact = df.groupby('segment')[['baseline_expected_revenue','post_strategy_expected_revenue']].sum()

segment_impact['uplift'] = (segment_impact['post_strategy_expected_revenue'] - segment_impact['baseline_expected_revenue'])

segment_impact

,baseline_expected_revenue,post_strategy_expected_revenue,uplift
segment,,,
0,40.650000,40.650000,0.000000
1,75.100000,67.590000,-7.510000
2,428927.983271,411328.096127,-17599.887144
3,73.801109,73.801109,0.000000


In [19]:
df.to_csv("../data/processed/churn_segmented.csv", index=False)